In [1]:
# test_final_api.ipynb

import numpy as np
import pandas as pd
import neural_mi # <-- Import the top-level library
from neural_mi.data.processors import ContinuousProcessor

if __name__ == '__main__':
    print(f"--- Final Integration Test: The Unified run() API ---")
    device = 'cpu'

    # --- 1. Generate Data ---
    x_raw = np.random.randn(20, 8000) # 20 channels for dimensionality
    y_raw = x_raw[:5, :] + np.random.randn(5, 8000) * 0.5 # Y has 5 channels
    
    cont_proc = ContinuousProcessor(window_size=100, step_size=50)
    x_data = cont_proc.process(x_raw).to(device)
    y_data = cont_proc.process(y_raw).to(device)
    print(f"Data shapes: X={x_data.shape}, Y={y_data.shape}")
    print("-" * 50)

    # --- 2. Define Base Parameters for all runs ---
    base_params = {
        'hidden_dim': 64, 'n_layers': 2, 'learning_rate': 1e-4,
        'n_epochs': 5, 'batch_size': 64, 'patience': 3
    }
    
    # --- Test 1: mode='estimate' ---
    print("\n--- Testing mode='estimate' ---")
    single_mi = neural_mi.run(x_data, y_data, mode='estimate', base_params=base_params, 
                              sweep_grid={'embedding_dim': [8]})
    print(f"Single MI estimate: {single_mi:.4f}")
    assert isinstance(single_mi, float)
    print("-" * 50)

    # --- Test 2: mode='sweep' ---
    print("\n--- Testing mode='sweep' ---")
    sweep_df = neural_mi.run(x_data, y_data, mode='sweep', base_params=base_params,
                             sweep_grid={'embedding_dim': [4, 8]})
    print("Sweep results:")
    print(sweep_df[['embedding_dim', 'test_mi']])
    assert isinstance(sweep_df, pd.DataFrame)
    assert len(sweep_df) == 2
    print("-" * 50)

    # --- Test 3: mode='dimensionality' ---
    print("\n--- Testing mode='dimensionality' ---")
    dim_df = neural_mi.run(x_data, mode='dimensionality', base_params=base_params,
                           sweep_grid={'embedding_dim': [4, 8]}, n_splits=2, n_workers=2)
    print("Dimensionality results:")
    print(dim_df[['embedding_dim', 'mi_mean']])
    assert isinstance(dim_df, pd.DataFrame)
    assert 'mi_mean' in dim_df.columns
    print("-" * 50)

    # --- Test 4: mode='rigorous' ---
    print("\n--- Testing mode='rigorous' ---")
    rigorous_results = neural_mi.run(x_data, y_data, mode='rigorous', base_params=base_params,
                                     sweep_grid={'embedding_dim': [8]}, gamma_range=range(1, 4), n_workers=2)
    print("Rigorous estimate results:")
    print(rigorous_results)
    assert isinstance(rigorous_results, list)
    assert 'mi_corrected' in rigorous_results[0]
    print("-" * 50)

    print("\n✅ All API modes tested successfully! The library is complete!")

--- Final Integration Test: The Unified run() API ---
Data shapes: X=torch.Size([159, 20, 100]), Y=torch.Size([159, 5, 100])
--------------------------------------------------

--- Testing mode='estimate' ---
Starting parameter sweep with 12 workers...
Created 1 tasks for the sweep...
Epoch 1/5 | Test MI: 0.0250
Epoch 2/5 | Test MI: 0.0266
Epoch 3/5 | Test MI: 0.0283
Epoch 4/5 | Test MI: 0.0301
Epoch 5/5 | Test MI: 0.0319
Best epoch identified (via smoothed curve): 5 (Smoothed MI: 0.0312)
Parameter sweep finished.
Single MI estimate: 0.0319
--------------------------------------------------

--- Testing mode='sweep' ---
Starting parameter sweep with 12 workers...
Created 2 tasks for the sweep...
Epoch 1/5 | Test MI: -0.0739
Epoch 2/5 | Test MI: -0.0737
Epoch 3/5 | Test MI: -0.0740
Epoch 4/5 | Test MI: -0.0746
Epoch 5/5 | Test MI: -0.0754
Early stopping triggered after 3 epochs.
Best epoch identified (via smoothed curve): 1 (Smoothed MI: -0.0740)
Epoch 1/5 | Test MI: -0.0134
Epoch 2/5 |

/Users/eabdelaleem3/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology/Postdoc Projects/NeuralMI/neural_mi/analysis/workflow.py:122: UserWarning: Fit for {'hidden_dim': np.int64(64), 'n_layers': np.int64(2), 'learning_rate': np.float64(0.0001), 'n_epochs': np.int64(5), 'batch_size': np.int64(64), 'patience': np.int64(3), 'critic_type': 'separable', 'estimator_fn': <function infonce_lower_bound at 0x317054680>, 'use_variational': np.False_, 'input_dim_x': np.int64(2000), 'input_dim_y': np.int64(500), 'embedding_dim': np.int64(8)} is unreliable (final gamma points < 5).
  warnings.warn(f"Fit for {param_dict} is unreliable (final gamma points < {min_gamma_points}).")
